**Problem 1**

In [1]:
from enum import Enum

b64_encoding_table = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijk" \
                     "lmnopqrstuvwxyz0123456789+/"


class Status(Enum):
    #Represents the possible status of the converter upon finishing to read a hex character of 4 bits.
   
    START_NEW = 0
    two= 1
    four = 2


def hex_to_base64(hexdata):
    """Returns a Base64 encoding of the given Hexadecimal string."""
    b64data = ""                # Encoding returned

    sixbits = 0                 # Group of six bits that being encoded
    status = Status.START_NEW   # Status of the conversion

    for hexchar in hexdata:
        dec = int(hexchar, 16)   # Decimal value of the character

        
        if status == Status.START_NEW:
            sixbits = dec
            status = Status.two

     
        elif status == Status.two:
            sixbits = (sixbits << 2) | (dec >> 2)
            b64data += b64_encoding_table[sixbits]
            sixbits = (dec & 0x3)   # 0x3 is 0011
            status = Status.four

            
            sixbits = (sixbits << 4) | dec
            b64data += b64_encoding_table[sixbits]
            status = Status.START_NEW

    
    if status == Status.two:
        sixbits <<= 2
        b64data += b64_encoding_table[sixbits]
        b64data += "="

    
    elif status == Status.four:
        sixbits <<= 4
        b64data += b64_encoding_table[sixbits]
        b64data += "=="

    return b64data


def main():
    assert hex_to_base64("49276d206b696c6c696e6720796f757220627261696e206c696b"
                         "65206120706f69736f6e6f7573206d757368726f6f6d") ==\
           "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
    
if __name__ == '__main__':
    main()
    
print(hex_to_base64("49276d206b696c6c696e6720796f757220627261696"
                    "e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"))
 

SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


**Problem 2**

In [2]:
def hex_xor(hexdata1, hexdata2):
    """Takes two equal-length hex encoded buffers and produces their XOR combination."""
    dec1 = int(hexdata1, 16)
    dec2 = int(hexdata2, 16)
    xor = dec1 ^ dec2
    return hex(xor)[2:]


def main():
     assert hex_xor("1c0111001f010100061a024b53535009181c", "686974207468652062756c6c277320657965") ==\
           "746865206b696420646f6e277420706c6179"

if __name__ == '__main__':
    main()
print(hex_xor("1c0111001f010100061a024b53535009181c", "686974207468652062756c6c277320657965"))


746865206b696420646f6e277420706c6179


 **Problem 3**

In [76]:
##Getting Error constantly
CHARACTER_FREQ = {
    'a': 0.0651738, 'b': 0.0124248, 'c': 0.0217339, 'd': 0.0349835, 'e': 0.1041442, 'f': 0.0197881, 'g': 0.0158610,
    'h': 0.0492888, 'i': 0.0558094, 'j': 0.0009033, 'k': 0.0050529, 'l': 0.0331490, 'm': 0.0202124, 'n': 0.0564513,
    'o': 0.0596302, 'p': 0.0137645, 'q': 0.0008606, 'r': 0.0497563, 's': 0.0515760, 't': 0.0729357, 'u': 0.0225134,
    'v': 0.0082903, 'w': 0.0171272, 'x': 0.0013692, 'y': 0.0145984, 'z': 0.0007836, ' ': 0.1918182
}


def get_english_score(input_bytes):
    
    score = 0

    for byte in input_bytes:
        score += CHARACTER_FREQ.get(chr(byte).lower(), 0)

    return score


def singlechar_xor(input_bytes, key_value):
    """XORs every byte of the input with the given key_value and returns the result."""
    output = b''

    for char in input_bytes:
        
        #return bytes(x ^ y for x, y in zip((input_bytes, key_value))
        
        if isinstance(char, str):
            char= chr(char)
        elif isinstance(char, int):
            char= chr(char)
        else:
        #if isinstance(input_bytes, str):
            # Text strings contain single characters
        
            output += chr((ord(a)^ord(b)) for a, b in zip(input_bytes, key_value))
            #output += chr((ord(char)^ord(key_value)))
            #return "".join(chr(ord(a) ^ ord(b)) for a, b in zip(s, t))
        
        #else :
            output += bytes([ord(char) ^ key_value])
                
      

         #output += bytes([char ^ key_value])
               
    return output


def singlechar_xor_brute_force(ciphertext):
    """Tries every possible byte for the single-char key, decrypts the ciphertext with that byte
    and computes the english score for each plaintext. The plaintext with the highest score
    is likely to be the one decrypted with the correct value of key.
    """
    candidates = []

    for k in range(256):
        key_candidate= [k]*len(ciphertext)
        plaintext_candidate = singlechar_xor(ciphertext, key_candidate)
        candidate_score = get_english_score(plaintext_candidate)

        result = {
            'key': key_candidate,
            'score': candidate_score,
            'plaintext': plaintext_candidate
        }

        candidates.append(result)

    # Return the candidate with the highest English score
    return sorted(candidates, key=lambda c: c['score'], reverse=True)[0]


def pretty_print_result(result):
    """Prints the given resulting candidate in a pretty format."""
    print(result['plaintext'].decode().rstrip(), "\tScore:", "{0:.2f}".format(result['score']),
          "\tKey:", chr(result['key']))



def main():
   
    ciphertext = bytes.fromhex("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")
    most_likely_plaintext = singlechar_xor_brute_force(ciphertext)
    pretty_print_result(most_likely_plaintext)

    
if __name__=='__main__':
    main()

#x = singlechar_xor_brute_force("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")
ciphertext = bytearray.fromhex("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")
most_likely_plaintext = singlechar_xor_brute_force(ciphertext)
print(pretty_print_result(most_likely_plaintext))

#print(most_likely_plaintext['plaintext'].rstrip())

TypeError: an integer is required (got type list)

**Problem 4**


In [21]:
def detect_encrypted_text(encrypted_strings):
    """Performs a singlechar XOR brute force attack to every ciphertext of the input, gets a plaintext
    from each of the ciphertexts and returns the decrypted plaintext which has the highest English score.
    """
    candidates = []

    for string in encrypted_strings:
        candidates.append(singlechar_xor_brute_force(string))

    # Return the candidate with the highest English score
    return sorted(candidates, key=lambda c: c['score'], reverse=True)[0]


def main():
    ciphertexts = [bytes.fromhex(line.strip()) for line in open("4.txt")]
    most_likely_plaintext = detect_encrypted_text(ciphertexts)
    pretty_print_result(most_likely_plaintext)

    # Check that the attack works properly
    assert most_likely_plaintext['plaintext'].rstrip() == b"Now that the party is jumping"

if __name__ == "__main__":
    main()

Now that the party is jumping 	Score: 2.09 	Key: 5


**Problem 5**

In [79]:
from binascii import hexlify


def repeating_key_xor(plaintext, key):
    """Implements the repeating-key XOR encryption."""
    ciphertext = b''
    i = 0

    for byte in plaintext:
        ciphertext += bytes([byte ^ key[i]])

        # Cycle i to point to the next byte of the key
        i = i + 1 if i < len(key) - 1 else 0

    return ciphertext


def main():
    c = repeating_key_xor(b"Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal", b'ICE')



if __name__ == "__main__":
    main()

print(str(hexlify(c), "utf-8"))

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f
